# 第4章: 形態素解析

> 夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をMeCabを使って形態素解析し，その結果をneko.txt.mecabというファイルに保存せよ．  
> このファイルを用いて，以下の問に対応するプログラムを実装せよ．
> 
> なお，問題37, 38, 39は[matplotlib](https://matplotlib.org/)もしくは[Gnuplot](http://www.gnuplot.info/)を用いるとよい．


解答はQiitaの[【言語処理100本ノック 2020】第4章: 形態素解析](https://qiita.com/yamaru/items/e06014b146a18e97ca59)を利用

In [ ]:
import os
import math
from collections import defaultdict 

import matplotlib.pyplot as plt
import japanize_matplotlib

In [ ]:
# データを保存するディレクトリの作成
DATADIR = "data"
CURRENTDIR = "/workspace/notebook"
CHAPDIR = os.path.join(DATADIR, "chapter4")

try:
    os.mkdir(CHAPDIR)
except:
    print("作成済み等の理由でディレクトリが作成されませんでした")

In [ ]:
# neko.txtの取得
!wget -P $CURRENTDIR/$CHAPDIR "https://nlp100.github.io/data/neko.txt"

In [ ]:
# ダウンロードしたテキストデータをMeCabで形態素に分解し、拡張子が「.mecab」のファイルとして出力する。
!mecab -o $CURRENTDIR/$CHAPDIR/neko.txt.mecab $CURRENTDIR/$CHAPDIR/neko.txt

In [ ]:
!wc -l $CURRENTDIR/$CHAPDIR/neko.txt.mecab

In [ ]:
!head -50 $CURRENTDIR/$CHAPDIR/neko.txt.mecab

In [ ]:
filename = "neko.txt.mecab"
in_fpath = os.path.join(CHAPDIR, filename)

## 30. 形態素解析結果の読み込み

> 形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．

In [ ]:
sentences = []
morphs = []
with open(in_fpath, mode="r") as f:
    for line in f:
        if line != "EOS\n": # 1文の終わりを判定する条件(今回は1文ごとに形態素情報(辞書型)を1つのリストにまとめるため)
            fields = line.split("\t")
            if len(fields) != 2 or fields[0] == "": # "\n"と"\t記号,一般,*,*,*,*,*\n"の行を除外するための条件
                continue
            else:
                attr =  fields[1].split(",")
                morph = {"surface": fields[0], "base": attr[6], "pos": attr[0], "pos1": attr[1]}
                morphs.append(morph)
        else: # 文末：形態素リストを文リストに追加
            sentences.append(morphs)
            morphs = []

for morph in sentences[2]:
    print(morph)

## 31. 動詞

> 動詞の表層形をすべて抽出せよ．

In [ ]:
verb_surfs = set()
for sentence in sentences:
    for morph in sentence:
        if morph["pos"] == "動詞":
            verb_surfs.add(morph["surface"])

print(f"動詞の表層形の種類: {len(verb_surfs)}\n")
print("---サンプル---")
for verb_surf in list(verb_surfs)[:10]:
    print(verb_surf)

## 32. 動詞の基本形

> 動詞の基本形をすべて抽出せよ．

In [ ]:
verb_bases = set()
for sentence in sentences:
    for morph in sentence:
        if morph["pos"] == "動詞":
            verb_bases.add(morph["base"])

print(f"動詞の基本形の種類: {len(verb_bases)}\n")
print("---サンプル---")
for verb_base in list(verb_bases)[:10]:
    print(verb_base)

## 33. 「AのB」

> 2つの名詞が「の」で連結されている名詞句を抽出せよ．

In [ ]:
anobs = set()
for sentence in sentences:
    for i in range(1, len(sentence) - 1):
        if sentence[i - 1]["pos"] == "名詞" and sentence[i]["surface"] == "の" and sentence[i + 1]["pos"] == "名詞":
            anobs.add("".join([sentence[i - 1]["surface"], sentence[i]["surface"], sentence[i + 1]["surface"]]))

print(f"「名詞+の+名詞」の種類: {len(anobs)}\n")
print("---サンプル---")
for n in list(anobs)[:10]:
    print(n)

## 34. 名詞の連接

> 名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．

In [ ]:
ans = set()
for sentence in sentences:
    nouns = ""
    num = 0
    for morph in sentence:
        if morph["pos"] == "名詞":  # 最初の形態素から順に、名詞であればnounsに連結し、連結数(num)をカウント
            nouns = "".join([nouns, morph["surface"]])
            num += 1
        elif num >= 2:  # 名詞以外、かつここまでの連結数が2以上の場合は出力し、nounsとnumを初期化
            ans.add(nouns)
            nouns = ""
            num = 0
        else:  # それ以外の場合、nounsとnumを初期化
            nouns = ""
            num = 0
    if num >= 2: 
        ans.add(nouns)

# 確認
print(f"連接名詞の種類: {len(ans)}\n")
print("---サンプル---")
for n in list(ans)[:10]:
    print(n)

## 35. 単語の出現頻度

> 文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．

In [ ]:
ans = defaultdict(int) 
for sentence in sentences:
    for morph in sentence:
        if morph["pos"] != "記号": 
            ans[morph["base"]] += 1  
ans = sorted(ans.items(), key=lambda x: x[1], reverse=True) 

for w in ans[:10]:
    print(w)

## 36. 頻度上位10語

> 出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

In [ ]:
# 前半は「35. 単語の出現頻度」と同様
ans = defaultdict(int)
for sentence in sentences:
    for morph in sentence:
        if morph["pos"] != "記号":
            ans[morph["base"]] += 1
ans = sorted(ans.items(), key=lambda x: x[1], reverse=True)


keys = [a[0] for a in ans[0:10]] 
values = [a[1] for a in ans[0:10]] 
plt.figure(figsize=(8, 4))
plt.bar(keys, values)
plt.show()

## 37. 「猫」と共起頻度の高い上位10語

> 「猫」とよく共起する（共起頻度が高い）10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

In [ ]:
ans = defaultdict(int)
for sentence in sentences:
    if "猫" in [morph["surface"] for morph in sentence]: 
        for morph in sentence:
            if morph["pos"] != "記号":
                ans[morph["base"]] += 1

del ans["猫"] # "猫"自身は含めない
ans = sorted(ans.items(), key=lambda x: x[1], reverse=True)


keys = [a[0] for a in ans[0:10]]
values = [a[1] for a in ans[0:10]]
plt.figure(figsize=(8, 4))
plt.bar(keys, values)
plt.show()

## 38. ヒストグラム

> 単語の出現頻度のヒストグラムを描け．ただし，横軸は出現頻度を表し，1から単語の出現頻度の最大値までの線形目盛とする．縦軸はx軸で示される出現頻度となった単語の異なり数（種類数）である．

In [ ]:
# 単語のカウント処理は「35. 単語の出現頻度」と同様
ans = defaultdict(int)
for sentence in sentences:
    for morph in sentence:
        if morph["pos"] != "記号":
            ans[morph["base"]] += 1
ans = ans.values() 

plt.figure(figsize=(8, 4))
plt.hist(ans, bins=100) 
plt.xlabel("出現頻度")
plt.ylabel("単語の種類数")
plt.show()

## 39. Zipfの法則

> 単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．

In [ ]:
# 単語のカウント処理は「35. 単語の出現頻度」と同様
ans = defaultdict(int)
for sentence in sentences:
    for morph in sentence:
        if morph["pos"] != "記号":
            ans[morph["base"]] += 1
ans = sorted(ans.items(), key=lambda x: x[1], reverse=True) 

ranks = [r + 1 for r in range(len(ans))] # 単語の出現頻度順位をリスト化（1～len(ans)）
values = [a[1] for a in ans] # 単語の出現頻度をリスト化
plt.figure(figsize=(8, 4))
plt.scatter(ranks, values)
plt.xscale("log") # ログスケール表示
plt.yscale("log") # ログスケール表示
plt.xlabel("出現頻度順位")
plt.ylabel("出現頻度")
plt.show()